# 0. Import libraries

The libraries used to create the data are in the following cell

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping

from modeling import * 
from constants import DATA_FOLDER_PATH

2023-09-27 11:43:01.186330: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-27 11:43:01.808733: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# 1. Define constants

This section defines constants to specify the location of the data and where the model will be stored

In [2]:
# Path to the files where the data for the training is
FEATURES_FILENAME = "surfaces_linear.npy"
LABELS_FILENAME = "coefficients_linear.npy"

FEATURES_FILE_PATH = f"{DATA_FOLDER_PATH}/{FEATURES_FILENAME}"
LABELS_FILE_PATH = f"{DATA_FOLDER_PATH}/{LABELS_FILENAME}"

# 2. Read data

Get train, validation and test data

In [3]:
train_features, \
train_labels, \
val_features, \
val_labels, \
test_features, \
test_labels = read_data_for_training(FEATURES_FILE_PATH,
                                     LABELS_FILE_PATH)

# 3. Define hyperparameters for neural network

In the first place define the hyperparameters of the neural network

In [4]:
# Neural network architecture hyperparameters
input_shape = train_features[0].shape
output_size = len(train_labels[0])
hidden_layer_sizes = [512, 256, 64]
regularizer = keras.regularizers.L1L2(l1=0.001,l2=0.1)
hidden_activation = 'relu'
output_activation = 'linear'
use_batch_normalization = True
name = 'TestModel'

# Compilation hyperparameters
loss_function = tf.keras.losses.MeanSquaredError()
learning_rate = 0.01
optimizer = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999)
metric = tf.keras.metrics.MeanSquaredError()

# Training hyperparameters
epochs = 1000
batch_size = 512
reduce_lr = ReduceLROnPlateau('mean_squared_error', factor=0.1, patience=150, verbose=1)
early_stop = EarlyStopping('mean_squared_error', patience=500, verbose=1)
callbacks = [reduce_lr, early_stop]

2023-09-27 11:43:02.742474: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 11:43:02.762340: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-27 11:43:02.762538: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

# 4. Train Network

Create the architecture

In [5]:
surface_reconstructor_model = create_linear_architecture(input_shape,
                                                         output_size,
                                                         hidden_layer_sizes,
                                                         regularizer,
                                                         hidden_activation,
                                                         output_activation,
                                                         use_batch_normalization=use_batch_normalization,
                                                         name=name)

Compile the model

In [6]:
compile_linear_model(surface_reconstructor_model,
                     loss_function,
                     optimizer,
                     metric
                     )

Fit the model

In [ ]:
history = train_linear_model(surface_reconstructor_model,
                             train_features,
                             train_labels,
                             batch_size,
                             epochs,
                             val_features,
                             val_labels,
                             callbacks)

Epoch 1/1000


2023-09-27 11:43:04.976637: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:606] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-09-27 11:43:04.979367: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb522fdcd30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-27 11:43:04.979388: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-09-27 11:43:04.983063: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-09-27 11:43:05.110851: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-27 11:43:05.209341: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifeti

94/94 [==============================] - 3s 8ms/step - loss: 28.1069 - mean_squared_error: 8.8133 - val_loss: 15.3895 - val_mean_squared_error: 11.9132 - lr: 0.0100
Epoch 2/1000
94/94 [==============================] - 1s 6ms/step - loss: 6.3070 - mean_squared_error: 3.5851 - val_loss: 11.5340 - val_mean_squared_error: 9.3432 - lr: 0.0100
Epoch 3/1000
94/94 [==============================] - 1s 6ms/step - loss: 5.0930 - mean_squared_error: 3.1619 - val_loss: 8.2340 - val_mean_squared_error: 6.5543 - lr: 0.0100
Epoch 4/1000
94/94 [==============================] - 1s 6ms/step - loss: 4.3722 - mean_squared_error: 2.8962 - val_loss: 5.6872 - val_mean_squared_error: 4.1118 - lr: 0.0100
Epoch 5/1000
94/94 [==============================] - 1s 6ms/step - loss: 3.8319 - mean_squared_error: 2.6497 - val_loss: 6.2196 - val_mean_squared_error: 5.1725 - lr: 0.0100
Epoch 6/1000
94/94 [==============================] - 1s 7ms/step - loss: 3.4983 - mean_squared_error: 2.5359 - val_loss: 4.9743 - val

Epoch 48/1000
94/94 [==============================] - 1s 6ms/step - loss: 2.4963 - mean_squared_error: 2.0091 - val_loss: 6.0418 - val_mean_squared_error: 5.5540 - lr: 0.0100
Epoch 49/1000
94/94 [==============================] - 1s 6ms/step - loss: 2.5152 - mean_squared_error: 2.0182 - val_loss: 7.7529 - val_mean_squared_error: 7.2504 - lr: 0.0100
Epoch 50/1000
94/94 [==============================] - 1s 7ms/step - loss: 2.5371 - mean_squared_error: 2.0332 - val_loss: 9.9167 - val_mean_squared_error: 9.3973 - lr: 0.0100
Epoch 51/1000
94/94 [==============================] - 1s 6ms/step - loss: 2.5297 - mean_squared_error: 2.0211 - val_loss: 12.7574 - val_mean_squared_error: 12.2340 - lr: 0.0100
Epoch 52/1000
94/94 [==============================] - 1s 7ms/step - loss: 2.5295 - mean_squared_error: 2.0151 - val_loss: 7.0604 - val_mean_squared_error: 6.5779 - lr: 0.0100
Epoch 53/1000
94/94 [==============================] - 1s 6ms/step - loss: 2.5497 - mean_squared_error: 2.0296 - val_l

94/94 [==============================] - 1s 7ms/step - loss: 2.5055 - mean_squared_error: 1.9867 - val_loss: 36.6760 - val_mean_squared_error: 36.1423 - lr: 0.0100
Epoch 95/1000
94/94 [==============================] - 1s 7ms/step - loss: 2.4873 - mean_squared_error: 1.9734 - val_loss: 10.8028 - val_mean_squared_error: 10.3066 - lr: 0.0100
Epoch 96/1000
94/94 [==============================] - 1s 7ms/step - loss: 2.4781 - mean_squared_error: 1.9711 - val_loss: 7.0525 - val_mean_squared_error: 6.5604 - lr: 0.0100
Epoch 97/1000
94/94 [==============================] - 1s 8ms/step - loss: 2.4802 - mean_squared_error: 1.9743 - val_loss: 27.5577 - val_mean_squared_error: 27.0648 - lr: 0.0100
Epoch 98/1000
94/94 [==============================] - 1s 7ms/step - loss: 2.4859 - mean_squared_error: 1.9824 - val_loss: 15.0408 - val_mean_squared_error: 14.5396 - lr: 0.0100
Epoch 99/1000
94/94 [==============================] - 1s 6ms/step - loss: 2.4661 - mean_squared_error: 1.9609 - val_loss: 18.

Epoch 140/1000
94/94 [==============================] - 1s 7ms/step - loss: 2.4762 - mean_squared_error: 1.9696 - val_loss: 40.8606 - val_mean_squared_error: 40.3508 - lr: 0.0100
Epoch 141/1000
94/94 [==============================] - 1s 8ms/step - loss: 2.4482 - mean_squared_error: 1.9498 - val_loss: 20.1108 - val_mean_squared_error: 19.6285 - lr: 0.0100
Epoch 142/1000
94/94 [==============================] - 1s 7ms/step - loss: 2.4358 - mean_squared_error: 1.9511 - val_loss: 16.5783 - val_mean_squared_error: 16.1010 - lr: 0.0100
Epoch 143/1000
94/94 [==============================] - 1s 8ms/step - loss: 2.4428 - mean_squared_error: 1.9468 - val_loss: 10.7896 - val_mean_squared_error: 10.3135 - lr: 0.0100
Epoch 144/1000
94/94 [==============================] - 1s 8ms/step - loss: 2.4345 - mean_squared_error: 1.9491 - val_loss: 10.0662 - val_mean_squared_error: 9.5990 - lr: 0.0100
Epoch 145/1000
94/94 [==============================] - 1s 8ms/step - loss: 2.4524 - mean_squared_error: 1

Epoch 186/1000
94/94 [==============================] - 1s 8ms/step - loss: 2.3770 - mean_squared_error: 1.9144 - val_loss: 28.0419 - val_mean_squared_error: 27.5878 - lr: 0.0100
Epoch 187/1000
94/94 [==============================] - 1s 7ms/step - loss: 2.3640 - mean_squared_error: 1.9089 - val_loss: 28.4220 - val_mean_squared_error: 27.9690 - lr: 0.0100
Epoch 188/1000
94/94 [==============================] - 1s 8ms/step - loss: 2.3603 - mean_squared_error: 1.9082 - val_loss: 28.9004 - val_mean_squared_error: 28.4471 - lr: 0.0100
Epoch 189/1000
94/94 [==============================] - 1s 8ms/step - loss: 2.3845 - mean_squared_error: 1.9226 - val_loss: 31.7655 - val_mean_squared_error: 31.3047 - lr: 0.0100
Epoch 190/1000
94/94 [==============================] - 1s 7ms/step - loss: 2.3605 - mean_squared_error: 1.9080 - val_loss: 22.6906 - val_mean_squared_error: 22.2380 - lr: 0.0100
Epoch 191/1000
94/94 [==============================] - 1s 7ms/step - loss: 2.3676 - mean_squared_error: 

Epoch 232/1000
94/94 [==============================] - 1s 8ms/step - loss: 2.3595 - mean_squared_error: 1.9093 - val_loss: 20.7627 - val_mean_squared_error: 20.2836 - lr: 0.0100
Epoch 233/1000
94/94 [==============================] - ETA: 0s - loss: 2.3620 - mean_squared_error: 1.9100

# 5. Check training process

In [ ]:
plot_model_history(history)

# 6. Save model and description

In [ ]:
description = f"# Neural network architecture hyperparameters \n\
input_size = train_features[0].shape \n\
output_size = len(train_labels[0]) \n\
hidden_layer_sizes = [512, 256, 64] \n\
regularizer = keras.regularizers.L1L2(l1=0.001,l2=0.1) \n\
hidden_activation = 'relu' \n\
output_activation = 'linear' \n\
use_batch_normalization = True \n\
name = 'LinearSurfaceReconstructor' \n\
# Compilation hyperparameters \n\
loss_function = tf.keras.losses.MeanSquaredError() \n\
learning_rate = 0.01 \n\
optimizer = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999) \n\
metric = tf.keras.metrics.MeanSquaredError() \n\
# Training hyperparameters \n\
epochs = 750 \n\
batch_size = 512 \n\
reduce_lr = ReduceLROnPlateau('mean_squared_error', factor=0.1, patience=150, verbose=1) \n\
early_stop = EarlyStopping('mean_squared_error', patience=500, verbose=1) \n\
callbacks = [reduce_lr, early_stop]"

store_model(surface_reconstructor_model,
            name,
            description)